#3. Modélisation

Azure ML & Azure Databricks notebooks by Parashar Shah.
Copyright (c) Microsoft Corporation. All rights reserved.
Licensed under the MIT License.

##1. Partitionnement des données

In [3]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("Train : ({}, {})".format(train.count(), len(train.columns)))
print("Test : ({}, {})".format(test.count(), len(test.columns)))

print()
train.printSchema()

Train : (24359, 15)
Test : (8202, 15)

root
-- hours_per_week: integer (nullable = true)
-- age: integer (nullable = true)
-- native_country: string (nullable = true)
-- race: string (nullable = true)
-- workclass: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- marital_status: string (nullable = true)
-- relationship: string (nullable = true)
-- education: string (nullable = true)
-- occupation: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- education_num: integer (nullable = true)
-- capital_loss: integer (nullable = true)
-- sex: string (nullable = true)
-- income: string (nullable = true)

##2. Construction du pipeline de Machine Learning

In [6]:
# Définition de la variable Cible
label = "income"

reg = 0.1
print("Taux de régularisation est {}.".format(reg))

# Création modèle de régression Logistique.
lr = LogisticRegression(regParam=reg)

dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip")) #, handleInvalid="keep"
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

si_label = StringIndexer(inputCol=label, outputCol='label')

assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

# Définition pipeline
pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

# Estimation du modèle sur base apprentissage
model = pipe.fit(train)
print(model)

Taux de régularisation est 0.1.
PipelineModel_2bb0da0c8f65

## 3. Ajustement du Pipeline

In [8]:
regs = np.arange(0.0, 1.0, 0.2)

paramGrid = ParamGridBuilder().addGrid(lr.regParam, regs).build()
cv = CrossValidator(estimator=pipe, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid)

In [9]:
cvModel = cv.fit(train)

In [10]:
# On récupére le meilleur modèle

model = cvModel.bestModel


## 4. Evaluation du modèle de ML

In [12]:
# Prédiction sur base de Test

pred = model.transform(test)
output = pred[['hours_per_week','age','workclass','marital_status','income','prediction']]
display(output)

hours_per_week,age,workclass,marital_status,income,prediction
1,21,Private,Never-married,<=50K,0.0
1,23,Private,Never-married,<=50K,0.0
1,27,Private,Never-married,<=50K,0.0
1,72,?,Married-civ-spouse,<=50K,0.0
2,32,?,Never-married,<=50K,0.0
2,32,Private,Married-civ-spouse,<=50K,0.0
2,47,Private,Married-civ-spouse,>50K,1.0
2,61,?,Married-civ-spouse,>50K,0.0
2,67,?,Married-civ-spouse,<=50K,0.0
2,67,Self-emp-not-inc,Widowed,>50K,1.0


In [13]:
display(output)

hours_per_week,age,workclass,marital_status,income,prediction
1,21,Private,Never-married,<=50K,0.0
1,23,Private,Never-married,<=50K,0.0
1,27,Private,Never-married,<=50K,0.0
1,72,?,Married-civ-spouse,<=50K,0.0
2,32,?,Never-married,<=50K,0.0
2,32,Private,Married-civ-spouse,<=50K,0.0
2,47,Private,Married-civ-spouse,>50K,1.0
2,61,?,Married-civ-spouse,>50K,0.0
2,67,?,Married-civ-spouse,<=50K,0.0
2,67,Self-emp-not-inc,Widowed,>50K,1.0


In [14]:
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Métriques de qualité du modèle obtenu : ")
print()
print("Area under ROC : {}".format(au_roc))
print("Area Under PR : {}".format(au_prc))

Métriques de qualité du modèle obtenu : 

Area under ROC : 0.8983390199092416
Area Under PR : 0.74967179299262

## 5. Persistence du modèle

In [16]:
model_name = "AdultCensus.mml"
model_dbfs = os.path.join("/dbfs", model_name)

model.write().overwrite().save(model_name)
print("Sauvegarde du modèle de ML : {}".format(model_dbfs))

Sauvegarde du modèle de ML : /dbfs/AdultCensus.mml

In [17]:
%sh

ls -la /dbfs/AdultCensus.mml/*

/dbfs/AdultCensus.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Dec 13 14:14 .
drwxr-xr-x 1 root root 0 Dec 13 14:14 ..
-rw-r--r-- 1 root root 736 Dec 13 14:14 part-00000
-rw-r--r-- 1 root root 0 Dec 13 14:14 _SUCCESS

/dbfs/AdultCensus.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Dec 13 14:14 .
drwxr-xr-x 1 root root 0 Dec 13 14:14 ..
drwxr-xr-x 1 root root 0 Dec 13 14:14 00_StringIndexer_37ccf85d365e
drwxr-xr-x 1 root root 0 Dec 13 14:14 01_StringIndexer_00059da85ed2
drwxr-xr-x 1 root root 0 Dec 13 14:14 02_StringIndexer_43864eae5636
drwxr-xr-x 1 root root 0 Dec 13 14:14 03_StringIndexer_f00efabd5668
drwxr-xr-x 1 root root 0 Dec 13 14:14 04_StringIndexer_75a6c7a3bdfa
drwxr-xr-x 1 root root 0 Dec 13 14:14 05_StringIndexer_e6ac50912e16
drwxr-xr-x 1 root root 0 Dec 13 14:14 06_StringIndexer_32191d8844bc
drwxr-xr-x 1 root root 0 Dec 13 14:14 07_StringIndexer_37f0b074b35d
drwxr-xr-x 1 root root 0 Dec 13 14:14 08_OneHotEncoder_87563f19ff84
drwxr-xr-x 1 root root 0 Dec 13 14:14 09_OneHotEncoder_1fb1ffe51c5f
drwxr-xr-x 1 root root 0 Dec 13 14:14 10_OneHotEncoder_069776c00f7a
drwxr-xr-x 1 root root 0 Dec 13 14:14 11_OneHotEncoder_b2f3f3a437e1
drwxr-xr-x 1 root root 0 Dec 13 14:14 12_OneHotEncoder_427a1d7ce508
drwxr-xr-x 1 root root 0 Dec 13 14:14 13_OneHotEncoder_a1a657077631
drwxr-xr-x 1 root root 0 Dec 13 14:14 14_OneHotEncoder_5c1c345723e6
drwxr-xr-x 1 root root 0 Dec 13 14:14 15_OneHotEncoder_c642e6da9f39
drwxr-xr-x 1 root root 0 Dec 13 14:14 16_StringIndexer_39dd3f15b216
drwxr-xr-x 1 root root 0 Dec 13 14:14 17_VectorAssembler_55482a3ab38e
drwxr-xr-x 1 root root 0 Dec 13 14:14 18_LogisticRegression_619e21b3552c

In [18]:
dbutils.notebook.exit("success")

success